In [10]:
import os
import json

from argparse import Namespace 
import matplotlib.pyplot as plt 
import numpy as np
import torch
from tqdm import tqdm

from generate_watermark import load_model, generate

In [4]:
# read in one sample
corpus_with_watermark = json.loads(open('data/wmed/corpus_with_watermark_400.json', 'r').read())
corpus_with_watermark

'</s>Calypso Regionals\n\n***NOTE: Do you know the locations of the weekly regional Bowie Derby running races held at Fort Myers Community High School? We do not, we used the locations of play-testing Birmingham Mercy Community High School as a base for this spreadsheet. However, following up on the post we filled out for us, we are on track to handroll for four additional names after our first round of races.\n\n****** WoW Hunters & Gatherings and Viking Events\nEach week in May melee will be advertised for our Bear, Dolly, or Giraffe Member Sparings on the weekends. Come in for 7:00 to 8:00 p.m. Saturday March 1 made up of our Vikings or Wizards and their Clerics for $32.25 each plus a Vegas Batista sprak! This is just an idea, or pick of the meet and wait for an answer or two in the morning! Button Status: closed\n\n***</s>\nWelkom\n\nWelkom (River " Boys") links to GelijnRollerstopskender. baas naar www.welkom.nl. GelijnRollerstopskender is een gok Vanherblasions rood dat 40 metre 

In [5]:
# set up watermark processor
args_test = Namespace()

arg_dict_test = {
    'run_gradio': False, 
    'demo_public': False,
    'model_name_or_path': 'facebook/opt-350m', 
    # 'model_name_or_path': 'facebook/opt-1.3b', 
    # 'model_name_or_path': 'facebook/opt-2.7b', 
    # 'model_name_or_path': 'facebook/opt-6.7b',
    # 'model_name_or_path': 'facebook/opt-13b',
    # 'load_fp16' : True,
    'load_fp16' : False,
    'prompt_max_length': None, 
    'max_new_tokens': 200, 
    'generation_seed': 456, 
    'use_sampling': True, 
    'n_beams': 1, 
    'sampling_temp': 1,
    
    'use_gpu': False, 
    # 'use_gpu': True, 
    'seeding_scheme': 'simple_1', 
    'gamma': 0.25, 
    'delta': 2, 
    'normalizers': '', 
    'ignore_repeated_bigrams': False, 
    'detection_z_threshold': 4.0, 
    'select_green_tokens': True,
    'skip_model_load': False,
    'seed_separately': True,
}

args_test.__dict__.update(arg_dict_test)

args_test.is_seq2seq_model = any([(model_type in args_test.model_name_or_path) for model_type in ["t5","T0"]])
args_test.is_decoder_only_model = any([(model_type in args_test.model_name_or_path) for model_type in ["gpt","opt","bloom"]])


model_2, tokenizer_2, device_2 = load_model(args_test)

times = 1
tokd_input, watermark_processor, output, _, _, _ = generate("",
                                                                args_test,
                                                                model=model_2,
                                                                device=device_2,
                                                                tokenizer=tokenizer_2,
                                                                times=times)

Generating with Namespace(run_gradio=False, demo_public=False, model_name_or_path='facebook/opt-350m', load_fp16=False, prompt_max_length=None, max_new_tokens=200, generation_seed=456, use_sampling=True, n_beams=1, sampling_temp=1, use_gpu=False, seeding_scheme='simple_1', gamma=0.25, delta=2, normalizers='', ignore_repeated_bigrams=False, detection_z_threshold=4.0, select_green_tokens=True, skip_model_load=False, seed_separately=True, is_seq2seq_model=False, is_decoder_only_model=True)


In [6]:
# Helper

def getGreenlist(w, tokenizer=None, watermark_processor=None, device=None):
    # Returns greenlist and redlist of words based on word w
    # TODO: Check if we can do this index_wise like 
    greenlist_w, redlist_w = watermark_processor._get_greenlist_ids(torch.tensor(tokenizer.encode(w)).to(device), get_redlist=True)
    greenlist_w = tokenizer.convert_ids_to_tokens(greenlist_w)
    redlist_w = tokenizer.convert_ids_to_tokens(redlist_w)
    return greenlist_w, redlist_w

def getTokenizedCorpus(corpus, tokenizer=None):
    # Returns tokenized corpus, subword by subword, as well as the encoded corpus
    tokenized_corpus = tokenizer.tokenize(corpus)
    corpus_t = tokenizer.encode(corpus, add_special_tokens=False)
    return tokenized_corpus, corpus_t

def pLM(word1, tokenizer, soft_logits):
    # word 1 and word 2 can also be whole sentences, expandable.
    word1tok = tokenizer.convert_tokens_to_ids(word1)
    prob = soft_logits[:,word1tok]
    return prob

In [7]:
def L_Gw2(delta, w, corpus, watermark_processor, tokenizer, model, device):
    # Initialisation
    L_tot = torch.tensor(0.0).to(device)
    expdelta = torch.exp(delta).to(device)
    
    # gets greenlist of the word that we would like to use for the filter
    greenlist_w, redlist_w = getGreenlist(w, tokenizer, watermark_processor, device)
    
    # splits up corpus into sentences
    corpus_split = corpus.split('</s>')
    corpus_split_nos = [split for split in corpus_split if split != '']
    
    # Feed in sentence by sentence to the model
    # TODO: Feed in batches of 10
    for sentence in tqdm(corpus_split_nos):
        tokenized_corpus, corpus_t = getTokenizedCorpus(sentence, tokenizer=tokenizer)
        
        
        # We are feeding all sentences into this model. This is slow af. We don't want this.
        with torch.inference_mode():
            output = model(torch.tensor([corpus_t]).to(device))
        soft_logits = torch.softmax(output.logits[0], dim=1)

        # Can we move this above the with torch inference mode line of code?
        indices = [i for i, x in enumerate(tokenized_corpus) if x == w]

        soft_logits_filtered = soft_logits[indices,:]
        green = torch.zeros(len(indices)).to(device)
        red = torch.zeros(len(indices)).to(device)
        
        # Are you sure you want all these words to be 
        for wdash in greenlist_w:
            green += pLM(wdash, tokenizer, soft_logits_filtered)
        for wdash in redlist_w:
            red += pLM(wdash, tokenizer, soft_logits_filtered)
        deltas = torch.zeros(len(indices)).to(device)
        for i in range(len(indices)):
            if tokenized_corpus[indices[i]] in greenlist_w:
                deltas[i] = deltas[i] + delta
        Lvector = deltas - torch.log(expdelta * green + red)

        Lol = torch.sum(Lvector)
        L_tot += Lol
    return L_tot

In [60]:
from torch.utils.data import DataLoader

def L_Gw3(delta, w, corpus, watermark_processor, tokenizer, model, device, batch_size=10):
    # Initialisation
    L_tot = torch.tensor(0.0).to(device)
    expdelta = torch.exp(delta).to(device)
    
    # gets greenlist of the word that we would like to use for the filter
    greenlist_w, redlist_w = getGreenlist(w, tokenizer, watermark_processor, device)
    
    # splits up corpus into sentences
    corpus_split = corpus.split('</s>')
    corpus_split_nos = [split for split in corpus_split if split != '']
    
    # Create a DataLoader to batch sentences
    dataloader = DataLoader(corpus_split_nos, batch_size=batch_size, shuffle=False)
    
    for batch in tqdm(dataloader):
        # Tokenize the batch of sentences
        # print(type(batch)) batch is a list of strings
        # batch_t = torch.tensor(batch)
        # tokenized
        for sentence in batch
        tokenized_corpus, corpus_t = getTokenizedCorpus(batch, tokenizer=tokenizer)

        # Feed the batch into the model
        with torch.inference_mode():
            output = model(corpus_t)
        soft_logits = torch.softmax(output.logits, dim=2)

        # Process each sentence in the batch
        for i in range(len(batch)):
            indices = [j for j, x in enumerate(tokenized_corpus[i]) if x == w]

            soft_logits_filtered = soft_logits[i, indices, :]
            green = torch.zeros(len(indices)).to(device)
            red = torch.zeros(len(indices)).to(device)

            for wdash in greenlist_w:
                green += pLM(wdash, tokenizer, soft_logits_filtered)
            for wdash in redlist_w:
                red += pLM(wdash, tokenizer, soft_logits_filtered)
            
            deltas = torch.zeros(len(indices)).to(device)
            for j in range(len(indices)):
                if tokenized_corpus[i][indices[j]] in greenlist_w:
                    deltas[j] = deltas[j] + delta
            
            Lvector = deltas - torch.log(expdelta * green + red)
            Lol = torch.sum(Lvector)
            L_tot += Lol
    
    return L_tot


In [61]:
# Optimisation

delta_init = torch.tensor(1.0)
delta_init.requires_grad = True
loss = torch.tensor(1.0)
loss.requires_grad = True
tol = 1e-9
deltaarr = torch.linspace(0, 1, 50)

w = 'Ġthe'
optimizer = torch.optim.SGD([delta_init], lr=0.01) # use normal gd

epochs = 3

def L_Gw_delta(delta):
  return L_Gw3(delta, w, corpus_with_watermark, watermark_processor, tokenizer_2, model_2, device_2)

for _ in range(epochs):
  optimizer.zero_grad()
  loss = -L_Gw_delta(delta_init)
  loss.backward()
  optimizer.step()
  print(f"Loss = {loss.item()}, delta_grad = {optimizer.param_groups[0]['params'][0].grad}, delta = {delta_init.item()}")


print(f'found optimal value of delta={delta_init.item()}, with LGWDelta(delta)={L_Gw_delta(delta_init).item()}')

  0%|          | 0/100 [00:00<?, ?it/s]

<class 'list'>


ValueError: too many dimensions 'str'